In [0]:
#unzipping clinicaltrial_2021.csv
dbutils.fs.ls("/FileStore/tables/")


Out[1]: [FileInfo(path='dbfs:/FileStore/tables/FaultDataset.csv', name='FaultDataset.csv', size=1703184, modificationTime=1678831181000),
 FileInfo(path='dbfs:/FileStore/tables/Occupancy_Detection_Data.csv', name='Occupancy_Detection_Data.csv', size=50968, modificationTime=1677677019000),
 FileInfo(path='dbfs:/FileStore/tables/TS021_2021_2.csv', name='TS021_2021_2.csv', size=497239, modificationTime=1679495199000),
 FileInfo(path='dbfs:/FileStore/tables/account-models/', name='account-models/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/accounts/', name='accounts/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/activations/', name='activations/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/activations.zip', name='activations.zip', size=8411369, modificationTime=1675257073000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2019.csv/', name='clinicaltrial_2019.csv/', size=0, modificationTime=0),
 FileInfo(path=

In [0]:

dbutils.fs.cp("/FileStore/tables/clinicaltrial_2021.zip", "file:/tmp/")

Out[2]: True

In [0]:
%sh
ls /tmp/

Rserv
Rtmp427GmZ
chauffeur-daemon-params
chauffeur-daemon.pid
chauffeur-env.sh
clinicaltrial_2021.zip
custom-spark.conf
driver-daemon-params
driver-daemon.pid
driver-env.sh
hsperfdata_root
pharma.zip
systemd-private-f8464012239144638f46229d0c3d0983-apache2.service-FOjcbf
systemd-private-f8464012239144638f46229d0c3d0983-ntp.service-XwGori
systemd-private-f8464012239144638f46229d0c3d0983-systemd-logind.service-6EFs6h
systemd-private-f8464012239144638f46229d0c3d0983-systemd-resolved.service-2rZqYe
tmp.ORPNyfDgU0


In [0]:
%sh

unzip -d /tmp/ /tmp/clinicaltrial_2021.zip

Archive:  /tmp/clinicaltrial_2021.zip
  inflating: /tmp/clinicaltrial_2021.csv  


In [0]:
%sh
ls /tmp/clinicaltrial_2021.csv


/tmp/clinicaltrial_2021.csv


In [0]:

dbutils.fs.mkdirs("FileStore/tables/clinicaltrial_2021.csv")


Out[6]: True

In [0]:

dbutils.fs.mv("file:/tmp/clinicaltrial_2021.csv", "/FileStore/tables/clinicaltrial_2021.csv", True)

Out[7]: True

In [0]:
dbutils.fs.ls("FileStore/tables/clinicaltrial_2021.csv/")

Out[8]: [FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2021.csv/clinicaltrial_2021.csv', name='clinicaltrial_2021.csv', size=50359696, modificationTime=1682539938000)]

In [0]:
len(dbutils.fs.ls("FileStore/tables/clinicaltrial_2021.csv/"))


Out[9]: 1

In [0]:
dbutils.fs.head("/FileStore/tables/clinicaltrial_2021.csv/clinicaltrial_2021.csv")



[Truncated to first 65536 bytes]
Out[10]: "Id|Sponsor|Status|Start|Completion|Type|Submission|Conditions|Interventions\r\nNCT02758028|The University of Hong Kong|Recruiting|Aug 2005|Nov 2021|Interventional|Apr 2016||\r\nNCT02751957|Duke University|Completed|Jul 2016|Jul 2020|Interventional|Apr 2016|Autistic Disorder,Autism Spectrum Disorder|\r\nNCT02758483|Universidade Federal do Rio de Janeiro|Completed|Mar 2017|Jan 2018|Interventional|Apr 2016|Diabetes Mellitus|\r\nNCT02759848|Istanbul Medeniyet University|Completed|Jan 2012|Dec 2014|Observational|May 2016|Tuberculosis,Lung Diseases,Pulmonary Disease|\r\nNCT02758860|University of Roma La Sapienza|Active, not recruiting|Jun 2016|Sep 2020|Observational [Patient Registry]|Apr 2016|Diverticular Diseases,Diverticulum,Diverticulosis|\r\nNCT02757209|Consorzio Futuro in Ricerca|Completed|Apr 2016|Jan 2018|Interventional|Apr 2016|Asthma|Fluticasone,Xhance,Budesonide,Formoterol Fumarate,Salmeterol Xinafoate\r\nNCT02752438|Ankara University|Unk

In [0]:
clinicaltrial_2021 = sc.textFile('/FileStore/tables/clinicaltrial_2021.csv/').map(lambda line: line.split("|"))
clinicaltrial_2021.take(2)



Out[11]: [['Id',
  'Sponsor',
  'Status',
  'Start',
  'Completion',
  'Type',
  'Submission',
  'Conditions',
  'Interventions'],
 ['NCT02758028',
  'The University of Hong Kong',
  'Recruiting',
  'Aug 2005',
  'Nov 2021',
  'Interventional',
  'Apr 2016',
  '',
  '']]

In [0]:
header = clinicaltrial_2021.first() # extracting the header
clinicaltrial_2021 = clinicaltrial_2021.filter(lambda x: x != header)

In [0]:
#ques 1: The number of studies in the dataset
clinicaltrial_2021.count()

Out[13]: 387261

In [0]:
#ques 2 : Types of studies and their frequencies

clinicaltrial_2021_type =clinicaltrial_2021.map(lambda x: (x[5], 1)).reduceByKey(lambda a, b: a + b)

clinicaltrial_2021_type = clinicaltrial_2021_type.sortBy(lambda x: x[1], ascending=False)
for row in clinicaltrial_2021_type.collect():
    print(row)


('Interventional', 301472)
('Observational', 77540)
('Observational [Patient Registry]', 8180)
('Expanded Access', 69)


In [0]:
#ques 3: Top 5 condition with their frequency

conditions_rdd = clinicaltrial_2021.flatMap(lambda x: x[7].split(","))
condition_count_rdd = conditions_rdd.map(lambda x: (x.strip(), 1))
count_by_condition_rdd = condition_count_rdd.reduceByKey(lambda a, b: a + b)
top_conditions_rdd = count_by_condition_rdd.takeOrdered(6, lambda x: -x[1])
top_conditions_rdd = filter(lambda x: x[1] != 65131, top_conditions_rdd)# filtering the first tuples because only top 5 conditions were required
for condition, count in top_conditions_rdd:
    print(condition, count)

Carcinoma 13389
Diabetes Mellitus 11080
Neoplasms 9371
Breast Neoplasms 8640
Syndrome 8032


In [0]:
#this is only for checking the output
from operator import add
filtered_rdd = clinicaltrial_2021.filter(lambda row: row[2] == 'Completed' and '2021' in row[4])
mapped_rdd = filtered_rdd.map(lambda row: (row[4][0:3], 1))
reduced_rdd = mapped_rdd.reduceByKey(add)
def month_string(month):
    return month
sorted_rdd = reduced_rdd.sortByKey(keyfunc=month_string)
result = sorted_rdd.collect()
for row in result:
    print(row[0], row[1])



Apr 967
Aug 700
Feb 934
Jan 1131
Jul 819
Jun 1094
Mar 1227
May 984
Oct 187
Sep 528


In [0]:
## Ques no 5 was done before ques no 4 because in 4 the Pharma.csv file needed to extract. After ques 5 the the ques 4 was solved

In [0]:
#ques 5: Plot number of completed studies each month in a given year
from operator import add
flatmapped_rdd = clinicaltrial_2021.flatMap(lambda row: [(row[4][0:3], 1)] if row[2] == 'Completed' and '2021' in row[4] else [])
reduced_rdd = flatmapped_rdd.reduceByKey(add)

#  a function to return the position of a month abbreviation in the list
def month_position(month):
    order= ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    return order.index(month)

sorted_rdd = reduced_rdd.sortBy(lambda x: month_position(x[0]))
result = sorted_rdd.collect()
for row in result:
    print(row[0], row[1])



Jan 1131
Feb 934
Mar 1227
Apr 967
May 984
Jun 1094
Jul 819
Aug 700
Sep 528
Oct 187


In [0]:
#unzipping pharma dataset
dbutils.fs.ls("/FileStore/tables/")


Out[19]: [FileInfo(path='dbfs:/FileStore/tables/FaultDataset.csv', name='FaultDataset.csv', size=1703184, modificationTime=1678831181000),
 FileInfo(path='dbfs:/FileStore/tables/Occupancy_Detection_Data.csv', name='Occupancy_Detection_Data.csv', size=50968, modificationTime=1677677019000),
 FileInfo(path='dbfs:/FileStore/tables/TS021_2021_2.csv', name='TS021_2021_2.csv', size=497239, modificationTime=1679495199000),
 FileInfo(path='dbfs:/FileStore/tables/account-models/', name='account-models/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/accounts/', name='accounts/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/activations/', name='activations/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/activations.zip', name='activations.zip', size=8411369, modificationTime=1675257073000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2019.csv/', name='clinicaltrial_2019.csv/', size=0, modificationTime=0),
 FileInfo(path

In [0]:
dbutils.fs.cp("/FileStore/tables/pharma.zip", "file:/tmp/")


Out[20]: True

In [0]:
%sh
ls /tmp/


Rserv
Rtmp427GmZ
chauffeur-daemon-params
chauffeur-daemon.pid
chauffeur-env.sh
clinicaltrial_2021.zip
custom-spark.conf
driver-daemon-params
driver-daemon.pid
driver-env.sh
hsperfdata_root
pharma.zip
systemd-private-f8464012239144638f46229d0c3d0983-apache2.service-FOjcbf
systemd-private-f8464012239144638f46229d0c3d0983-ntp.service-XwGori
systemd-private-f8464012239144638f46229d0c3d0983-systemd-logind.service-6EFs6h
systemd-private-f8464012239144638f46229d0c3d0983-systemd-resolved.service-2rZqYe
tmp.ORPNyfDgU0


In [0]:
%sh
unzip -d /tmp/ /tmp/pharma.zip

Archive:  /tmp/pharma.zip
  inflating: /tmp/pharma.csv         


In [0]:
%sh
ls /tmp/pharma.csv


/tmp/pharma.csv


In [0]:
dbutils.fs.mkdirs("FileStore/tables/pharma.csv")


Out[24]: True

In [0]:
dbutils.fs.mv("file:/tmp/pharma.csv", "/FileStore/tables/pharma.csv", True)


Out[25]: True

In [0]:
dbutils.fs.ls("FileStore/tables/pharma.csv/")

Out[26]: [FileInfo(path='dbfs:/FileStore/tables/pharma.csv/pharma.csv', name='pharma.csv', size=678999, modificationTime=1682539964000)]

In [0]:
len(dbutils.fs.ls("FileStore/tables/pharma.csv/"))


Out[27]: 1

In [0]:
dbutils.fs.head("FileStore/tables/pharma.csv/pharma.csv")

[Truncated to first 65536 bytes]
Out[28]: '"Company","Parent_Company","Penalty_Amount","Subtraction_From_Penalty","Penalty_Amount_Adjusted_For_Eliminating_Multiple_Counting","Penalty_Year","Penalty_Date","Offense_Group","Primary_Offense","Secondary_Offense","Description","Level_of_Government","Action_Type","Agency","Civil/Criminal","Prosecution_Agreement","Court","Case_ID","Private_Litigation_Case_Title","Lawsuit_Resolution","Facility_State","City","Address","Zip","NAICS_Code","NAICS_Translation","HQ_Country_of_Parent","HQ_State_of_Parent","Ownership_Structure","Parent_Company_Stock_Ticker","Major_Industry_of_Parent","Specific_Industry_of_Parent","Info_Source","Notes"\n"Abbott Laboratories","Abbott Laboratories","$5,475,000","$0","$5,475,000","2013","20131227","government-contracting-related offenses","False Claims Act and related","kickbacks and bribery","Abbott Laboratories agreed to $5.475 million to resolve allegations that it violated the False Claims Act by paying kickbacks to in

In [0]:

pharma = sc.textFile('/FileStore/tables/pharma.csv/').map(lambda line: line.split("\",\""))
pharma.take(2)

Out[29]: [['"Company',
  'Parent_Company',
  'Penalty_Amount',
  'Subtraction_From_Penalty',
  'Penalty_Amount_Adjusted_For_Eliminating_Multiple_Counting',
  'Penalty_Year',
  'Penalty_Date',
  'Offense_Group',
  'Primary_Offense',
  'Secondary_Offense',
  'Description',
  'Level_of_Government',
  'Action_Type',
  'Agency',
  'Civil/Criminal',
  'Prosecution_Agreement',
  'Court',
  'Case_ID',
  'Private_Litigation_Case_Title',
  'Lawsuit_Resolution',
  'Facility_State',
  'City',
  'Address',
  'Zip',
  'NAICS_Code',
  'NAICS_Translation',
  'HQ_Country_of_Parent',
  'HQ_State_of_Parent',
  'Ownership_Structure',
  'Parent_Company_Stock_Ticker',
  'Major_Industry_of_Parent',
  'Specific_Industry_of_Parent',
  'Info_Source',
  'Notes"'],
 ['"Abbott Laboratories',
  'Abbott Laboratories',
  '$5,475,000',
  '$0',
  '$5,475,000',
  '2013',
  '20131227',
  'government-contracting-related offenses',
  'False Claims Act and related',
  'kickbacks and bribery',
  "Abbott Laboratories agreed t

In [0]:
pharma.count()

Out[30]: 969

In [0]:
#ques 4:10 most common sponsors that are not pharmaceutical companies, along with the number of clinical trials they have sponsored
parent_company_set = set(pharma.map(lambda row: row[1]).distinct().collect())
clinicaltrial_2021 = clinicaltrial_2021.filter(lambda row: row[1] not in parent_company_set)
mapped_rdd = clinicaltrial_2021.map(lambda row: (row[1], 1))
reduced_rdd = mapped_rdd.reduceByKey(lambda a, b: a + b)
sorted_rdd = reduced_rdd.sortBy(lambda row: row[1], False)
top_sponsors = sorted_rdd.take(10)# showing top 10
for sponsor, count in top_sponsors:
    print(sponsor, count)


National Cancer Institute (NCI) 3218
M.D. Anderson Cancer Center 2414
Assistance Publique - Hôpitaux de Paris 2369
Mayo Clinic 2300
Merck Sharp & Dohme Corp. 2243
Assiut University 2154
Novartis Pharmaceuticals 2088
Massachusetts General Hospital 1971
Cairo University 1928
Hoffmann-La Roche 1828
